# Libraries

In [26]:
import pandas as pd
from datetime import datetime
pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format

# Parameters

In [27]:
today = datetime.today()
year = today.year
week = today.isocalendar()[1]

#year = 2022
#week = 10

# Import Data

In [28]:
path = r'\\NASPRO.infovisiontv.com\DWH01\Cognos\Genomma lab USA\US Data\VIP Suerox\{0}\W {1} {0}' #r-string
path = path.format(year, str(week).zfill(2))
filename = '{0}_{1}_VIP.xlsx'
filename = filename.format(year, str(week).zfill(2))

In [30]:
df = pd.read_excel(path + '\\' + filename)

# Data Cleaning

In [31]:
report = df.pivot_table(index='Distributors', 
                        values=df.columns[-3:], 
                        aggfunc='sum', ).reset_index()

In [32]:
report.columns = ['Client', 'Cases', 'Dollars', 'Units']

In [33]:
mask = {'CVS MAINLAND':'CVS MAINLAND', 
        'CVS PR':'CVS PR', 
        'Vaqueria Tres Monjitas':'Vaqueria Tres Monjitas'}

report['Distributors'] = report['Client'].map(mask)

report['Distributors'].fillna('DSD USA', inplace=True)

# Report Final

In [34]:
final = report.pivot_table(index='Distributors', 
                           values=['Cases', 'Units', 'Dollars'],
                           aggfunc='sum').reset_index()

In [35]:
final['Cases'] = final['Cases'].map(int)

final['Week'] = week

# Export

In [36]:
final = final[['Week', 'Distributors', 'Cases', 'Units', 'Dollars']]

final.to_excel(path + '\Figures Validate {0}_{1}.xlsx'.format(year, str(week).zfill(2)), index=False)

# Graphs

## Import Last 4 Weeks

In [37]:
weeks = [w for w in range(week-3, week + 1)]

In [38]:
last_fourw = pd.DataFrame()
for week in weeks:
    path = r'\\NASPRO.infovisiontv.com\DWH01\Cognos\Genomma lab USA\US Data\VIP Suerox\{0}\W {1} {0}'
    path = path.format(year, str(week).zfill(2))
    filename = r'\\Figures Validate {0}_{1}.xlsx'
    filename = filename.format(year, str(week).zfill(2))
    last_fourw = pd.concat([last_fourw, pd.read_excel(path + filename)])

## Data Cleaning

In [39]:
last_fourw.dropna(axis=0, inplace=True)
last_fourw['Week'] = last_fourw['Week'].map(int)
last_fourw.reset_index(drop=True, inplace=True)

In [40]:
import plotly.express as px

In [41]:
fig = px.bar(last_fourw, x="Week", y="Units", color="Distributors", text_auto=True)
fig.show()

No se está viendo bien las cifras de CVS PR, tal vez este gráfico no sea el apropiado para mostrar esta información.

In [42]:
import altair as alt

In [43]:
bars = alt.Chart(last_fourw).mark_bar().encode(
    x=alt.X('sum(Dollars):Q', stack='zero'),
    y=alt.Y('Week:N'),
    color=alt.Color('Distributors')
)

text = alt.Chart(last_fourw).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('sum(Dollars):Q', stack='zero'),
    y=alt.Y('Week:N'),
    detail='Distributors:N',
    text=alt.Text('sum(Dollars):Q', format='.1f')
)

#text = alt.Order('color_Category_sort_index:Q')

bars + text

alt.LayerChart(...)

In [44]:
last_fourw

,Week,Distributors,Cases,Units,Dollars
0,8,CVS MAINLAND,494,5939,"8,611.55"
1,8,CVS PR,27,332,481.40
2,8,DSD USA,2301,27612,"38,656.80"
3,8,Vaqueria Tres Monjitas,341,4103,"5,744.20"
4,9,CVS MAINLAND,406,4881,"7,077.45"
5,9,CVS PR,31,373,540.85
6,9,DSD USA,1774,21288,"29,803.20"
7,9,Vaqueria Tres Monjitas,1571,18853,"26,394.20"
8,10,CVS MAINLAND,409,4909,"7,118.05"
9,10,CVS PR,27,327,474.15


In [45]:
aux = last_fourw[last_fourw['Distributors']=='CVS PR']
bars_cvs_pr = alt.Chart(aux).mark_bar().encode(
    x=alt.X('Week:N'),
    y=alt.Y('sum(Dollars):Q')
)

In [46]:
aux = last_fourw[last_fourw['Distributors']=='CVS MAINLAND']
bars_cvs_ml = alt.Chart(aux).mark_bar().encode(
    x=alt.X('Week:N'),
    y=alt.Y('sum(Dollars):Q')
)

In [47]:
aux = last_fourw[last_fourw['Distributors']=='Vaqueria Tres Monjitas']
bars_vtm = alt.Chart(aux).mark_bar().encode(
    x=alt.X('Week:N'),
    y=alt.Y('sum(Dollars):Q')
)

In [48]:
aux = last_fourw[last_fourw['Distributors']=='DSD USA']
bars_dsd = alt.Chart(aux).mark_bar().encode(
    x=alt.X('Week:N', angle=0),
    y=alt.Y('sum(Dollars):Q')
)

In [49]:
text = bars_dsd.mark_text(
    align='left',
    baseline='middle',
    dy=-5,
    dx=-18,
    fontSize=8,
).encode(
    text=alt.Text('sum(Dollars):Q', format="$,.1f")
)

In [50]:
?alt.X

Init signature:
alt.X(
    shorthand=Undefined,
    aggregate=Undefined,
    axis=Undefined,
    band=Undefined,
    bin=Undefined,
    field=Undefined,
    impute=Undefined,
    scale=Undefined,
    sort=Undefined,
    stack=Undefined,
    timeUnit=Undefined,
    title=Undefined,
    type=Undefined,
    **kwds,
)
Docstring:     
X schema wrapper

Mapping(required=[shorthand])

Attributes
----------

shorthand : string
    shorthand for field, aggregate, and type
aggregate : :class:`Aggregate`
    Aggregation function for the field
    (e.g., ``"mean"``, ``"sum"``, ``"median"``, ``"min"``, ``"max"``, ``"count"`` ).

    **Default value:** ``undefined`` (None)

    **See also:** `aggregate <https://vega.github.io/vega-lite/docs/aggregate.html>`__
    documentation.
axis : anyOf(:class:`Axis`, None)
    An object defining properties of axis's gridlines, ticks and labels.
    If ``null``, the axis for the encoding channel will be removed.

    **Default value:** If undefined, default `axi

In [51]:
(bars_dsd + text).properties(width=170, height=300)

SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.channels.X, validating 'additionalProperties'

        Additional properties are not allowed ('angle' was unexpected)
        

alt.LayerChart(...)

In [ ]:
bars_cvs_pr | bars_cvs_ml | bars_vtm | bars_dsd